In [1]:
import os, sys, mimetypes, io, pandas as pd, numpy, mimetypes
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_columns", None)

In [2]:
class FormatClassifier():
    def __init__(self, discard=None):
        self.content=pd.DataFrame()
        self.mimetype={}
        #Hyper parameter(s)
        self.discard = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-_ ' if discard is None else discard

    def _preprocess(self, filename=None, string=None):
        """ Get  char frequency """
       
        #Char count
        if filename is not None: 
            with open(filename, 'r+') as f: string = f.read(50000)
        assert string is not None
        count = {}
        wc, r = 0, False
        for c in string:
            if c in self.discard: 
                if not r: (r, wc) = (True, wc+1)
            else: 
                count[c] = count[c]+1 if c in count else 1
                r=False
        #Char frequency
        size = sum(count.values()  )
        count['density'] = size/wc if wc != 0 else 100 #len(string)
        glyph = lambda c, non_printable = {'\n':'CR', '\t':'TAB', ' ': 'SP'}: non_printable[c] if c in non_printable else c     #Quick-win to display special char
        return { glyph(k):round(v/size,4)*100 for k,v in count.items() } if size !=0 else {}
    
    #Retrieve frequency of special chars for each file type
    def train(self, directory):
        
        #Loop examples
        all_examples, d= [], os.path.sep
        for filetype in os.listdir( directory ):
            if not os.path.isdir( directory+d+filetype ): continue
            for example in os.listdir( directory+d+filetype ):
                filename = directory+d+filetype+d+example
                
                #Use mimetype for binary files
                mimetype, encoding = mimetypes.guess_type(filename)
                if mimetype is not None and mimetype != 'text/plain':
                    if mimetype in self.mimetype: assert self.mimetype[mimetype] == filetype, self.mimetype
                    else: 
                        self.mimetype[mimetype] = filetype
                        continue
                        
                #Pre-processing for text file
                entry = self._preprocess( filename=filename  )
                #Add expected category for supervision purpose
                entry.update( {'filetype': filetype, 'filename': filename})
                all_examples.append(  entry )

        #Featurize: X = columns, Y = rows['filetype'] -- filename is kept only for debugging
        df = pd.DataFrame(all_examples).fillna(0).set_index(['filetype', 'filename'])
                
        #Store result
        self.content = df
        
        return df
    
    def estimate(self, string=None, filename=None, debug=False):
        if filename is not None:
            mimetype, encoding = mimetypes.guess_type(filename)
            if mimetype is not None and mimetype != 'text/plain': return self.mimetype[mimetype]
        #Preprocess
        x = self._preprocess(filename=filename, string=string)
        #Pseudo-Featurization
        x = pd.Series( [ x[k] if k in x else 0 for k in self.content.columns.values] , index= self.content.columns.values )
        #Estimate combinations
        df = pd.DataFrame([ (filetype, filename,  numpy.linalg.norm(x-row)+1, *(x-row).values ) for (filetype, filename), row in self.content.iterrows() ] )
        #Rank estimation results
        df = df.sort_values(by=[2], ascending=True)
        
        #Return estimation(s)
        if debug:
            df.columns = ['filetype', 'filename', 'distance'] + [ i for i in self.content.columns.values]
            return df.replace(0,'-').set_index(['filetype', 'filename'])
        else:
            return df.iloc[0,0]


In [5]:
model=  FormatClassifier(discard=None)
df=model.train( r'C:\_data\Lubies\Python\alita\examples')
#model.estimate( r'C:\_data\tmp\toto.txt', debug=True)
debug = model.estimate( string='http:toto:elel\nelke:llklk:', debug=True)
#model.estimate( filename=r'C:\_data\Lubies\Python\alita\examples\bin_excel\excel_blank.xlsx')
debug.head(20)
#model.estimate(  r'C:\_data\tmp\empty.txt'  )
# df, x_encode, x_decode, y_encode, y_decode = model._encode(['xml','ini'])

distance  \
filetype    filename                                                        
csv_column  C:\_data\Lubies\Python\alita\examples\csv_colum...  18.140000   
address_ip  C:\_data\Lubies\Python\alita\examples\address_i...  30.261750   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...  67.635868   
text_infa   C:\_data\Lubies\Python\alita\examples\text_infa...  72.943349   
            C:\_data\Lubies\Python\alita\examples\text_infa...  73.150132   
text_json   C:\_data\Lubies\Python\alita\examples\text_json...  80.763658   
address_url C:\_data\Lubies\Python\alita\examples\address_u...  81.779437   
text_json   C:\_data\Lubies\Python\alita\examples\text_json...  85.031311   
            C:\_data\Lubies\Python\alita\examples\text_json...  86.464245   
            C:\_data\Lubies\Python\alita\examples\text_json...  86.496533   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...  88.077077   
text_ini    C:\_data\Lubies\Python\alita\examples\text_ini\...  88.874050   
csv_column  C:\_data\Lubies\Python\alita\examples\csv_colum...  89.093395   
            C:\_data\Lubies\Python\alita\examples\csv_colum...  89.093395   
text_ini    C:\_data\Lubies\Python\alita\examples\text_ini\...  89.106916   
address_url C:\_data\Lubies\Python\alita\examples\address_u...  89.980831   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...  90.611353   
address_url C:\_data\Lubies\Python\alita\examples\address_u...  91.264565   
text_xml    C:\_data\Lubies\Python\alita\examples\text_xml\...  91.302260   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...  91.313441   

                                                                    "     #  \
filetype    filename                                                          
csv_column  C:\_data\Lubies\Python\alita\examples\csv_colum...      -     -   
address_ip  C:\_data\Lubies\Python\alita\examples\address_i...      -     -   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...  -6.45     -   
text_infa   C:\_data\Lubies\Python\alita\examples\text_infa...      -     -   
            C:\_data\Lubies\Python\alita\examples\text_infa...      -     -   
text_json   C:\_data\Lubies\Python\alita\examples\text_json... -29.41     -   
address_url C:\_data\Lubies\Python\alita\examples\address_u...      -     -   
text_json   C:\_data\Lubies\Python\alita\examples\text_json... -44.19     -   
            C:\_data\Lubies\Python\alita\examples\text_json... -44.44     -   
            C:\_data\Lubies\Python\alita\examples\text_json... -44.83     -   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...      -     -   
text_ini    C:\_data\Lubies\Python\alita\examples\text_ini\...      -     -   
csv_column  C:\_data\Lubies\Python\alita\examples\csv_colum...      -     -   
            C:\_data\Lubies\Python\alita\examples\csv_colum... -66.67     -   
text_ini    C:\_data\Lubies\Python\alita\examples\text_ini\...  -3.85     -   
address_url C:\_data\Lubies\Python\alita\examples\address_u...      -     -   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...      -     -   
address_url C:\_data\Lubies\Python\alita\examples\address_u...      -     -   
text_xml    C:\_data\Lubies\Python\alita\examples\text_xml\... -28.92 -0.31   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...      -     -   

                                                                    $      &  \
filetype    filename                                                           
csv_column  C:\_data\Lubies\Python\alita\examples\csv_colum...      -      -   
address_ip  C:\_data\Lubies\Python\alita\examples\address_i...      -      -   
text_sql    C:\_data\Lubies\Python\alita\examples\text_sql\...      -      -   
text_infa   C:\_data\Lubies\Python\alita\examples\text_infa... -18.37      -   
            C:\_data\Lubies\Python\alita\examples\text_infa... -17.48      -   
text_json   C:\_data\Lubies\Python\alita\examples\text_json...      -  

In [ ]:
model.content